# Notebook used to modify the 2023 questions and to produce translation

# 2023 Data Preparation and Translation

This notebook takes our 2023 survey materials and transforms them into a single, analysis-ready dataset. We:

1. Load the cleaned 2023 CodeBook and the raw GSB 2023 responses from Excel files  
2. Clean and merge both sources into one comprehensive pandas DataFrame  
3. Insert a placeholder message where Romanche translations are not yet available  
4. Export the final combined table to `combined_df.csv` for downstream analysis  

With these steps, we ensure all questions and responses are aligned, and any missing translations are clearly flagged before moving on to deeper analysis.


In [ ]:
import pandas as pd
import json

In [ ]:
# Load the data
df_codebook_23 = pd.read_excel("../data/Extraction CodeBook - 3. Cleaned.xlsx", sheet_name="2023")
df_gsb_23 = pd.read_excel("../data/GSB 2023_V1.xlsx")
df_gsb_other_years = pd.read_csv("../data/GSB_1988_2017_V1.csv")
df_qg = pd.read_excel("../data/QuestionGlobales.xlsx")


In [ ]:
df_qg.head(10)

In [ ]:
df_codebook_23.head(50)

In [ ]:
df_gsb_23.head()

In [ ]:
print(df_gsb_23.columns)

In [ ]:
df_gsb_23.head(4)
df_gsb_23[df_gsb_23["BFS_2023"] == 5586]

In [ ]:
df_gsb_23.shape

data merging 

In [ ]:
df_base_questions = df_codebook_23[df_codebook_23['code'].str.contains(r'GSB23_Q\d+\b')]
base_question_ids = df_base_questions['code'].str.extract(r'GSB23_Q(\d+)')[0].astype(int).unique()
df_suffix_questions = df_codebook_23[df_codebook_23['code'].str.contains(r'GSB23_Q\d+_1\b')]
for question_id in base_question_ids:
    df_suffix_questions = df_suffix_questions[~df_suffix_questions['code'].str.contains(f'GSB23_Q{question_id}_1\b')]
df_first_subquestions = pd.concat([df_base_questions, df_suffix_questions], ignore_index=True)
df_first_subquestions.reset_index(drop=True, inplace=True)

df_first_subquestions[df_first_subquestions['num_question'] == 'Q1']

In [ ]:
df_first_subquestions[df_first_subquestions["enquete"] == "GSB23"]

In [ ]:
df_first_subquestions.shape

In [ ]:
# Step 1: Liste des questions supplémentaires à ajouter
extra_questions_codes = [
    'GSB23_Teilnahme', 'GSB23_Mode', 'GSB23_StartDate', 'GSB23_EndDate', 
    'GSB23_Progress', 'GSB23_Duration', 'GSB23_Finished', 'GSB23_Recorded', 
    'GSB23_UserLang', 'GSB23_Q99', 'GSB23_Q100', 'GSB23_Q101', 'GSB23_UserLanguage'
]

# Step 2: Filtrer le df_codebook pour ces questions
df_extra_questions = df_codebook_23[df_codebook_23['code'].isin(extra_questions_codes)]

# Step 3: Combiner les 10 premières sous-questions avec les questions supplémentaires
df_combined = pd.concat([df_first_subquestions, df_extra_questions])

# Step 4: Réinitialiser les index du DataFrame combiné
df_combined.reset_index(drop=True, inplace=True)

# Step 5: Afficher le DataFrame combiné
df_combined.head(25)

In [ ]:
df_combined.shape

In [ ]:
df_combined.columns

In [ ]:
with open('../data/municipalities.json', encoding="utf-8") as f:
    municipalities_data = json.load(f)

list_of_commune_ids = [feature['properties']['id'] for feature in municipalities_data['features']]
#print(list_of_commune_ids)

# the column GSB23_Q100 contains the commune ids
df_filtered = df_gsb_23[df_gsb_23['GSB23_Q100'].isin(list_of_commune_ids)]

col_of_interest = df_combined['original_code'].tolist()

print(col_of_interest)
# Filtrer seulement les colonnes qui existent dans df_filtered
col_of_interest_filtered = [col for col in col_of_interest if col in df_filtered.columns]

df_commune_responses = df_filtered[col_of_interest_filtered]
df_commune_responses.head()

In [ ]:
df_commune_responses = df_commune_responses.loc[:, ~df_commune_responses.columns.duplicated()]

In [ ]:
print(df_commune_responses.columns)


In [ ]:
for feature in municipalities_data['features']:
    commune_id = feature['properties']['id']
    
    # Filtrer les réponses pour cette commune
    commune_response = df_commune_responses[df_commune_responses['GSB23_Q100'] == commune_id]
    
    if not commune_response.empty:
        # Ajouter la réponse au GeoJSON (par exemple pour une question)
        feature['properties']['response_Q1'] = commune_response['GSB23_Q1_1'].values[0]

df_commune_responses.to_csv('../data/commune_responses.csv', index=False, encoding='utf-8')

## Translation of the global question using an open source API


### Libretranslate -> not working locally 

### Bert model to translate 

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
df_combined['text_de'] = df_combined['label'].str.replace(r'^\d+\.\s*', '', regex=True)


model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translation function with mbart 
def translate_mbart(text, source_lang, target_lang):
    tokenizer.src_lang = source_lang
    # encode the text to be translated
    encoded_input = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    generated_tokens = model.generate(
        **encoded_input,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang],
        max_length=512,
        num_beams=4,  # use beam search for better results
        early_stopping=True
    )
    # decode the generated tokens
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

df_combined['text_en'] = df_combined['text_de'].apply(lambda x: translate_mbart(x, "de_DE", "en_XX"))
df_combined['text_fr'] = df_combined['text_en'].apply(lambda x: translate_mbart(x, "en_XX", "fr_XX"))
df_combined['text_it'] = df_combined['text_en'].apply(lambda x: translate_mbart(x, "en_XX", "it_IT"))

In [ ]:
df_combined.head()

In [ ]:
df_combined['text_ro'] = 'Translation not available for the moment'

In [ ]:
df_combined.to_csv('../data/combined_df.csv', index=False, encoding='utf-8')